In [ ]:
# Импорт необходимых модулей 
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
TEXT_COLOR = 'black'

matplotlib.rcParams['figure.figsize'] = (15, 10)
matplotlib.rcParams['text.color'] = 'black'
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['axes.labelcolor'] = TEXT_COLOR
matplotlib.rcParams['xtick.color'] = TEXT_COLOR
matplotlib.rcParams['ytick.color'] = TEXT_COLOR

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)

# Градиентный бустинг

По определению бустинг является еще одним методом ансамблирования, как и бэггинг. Отличием является принцип построения и, соответственно, работы модели. В бэггинге мы нехитро разделили набор данных и параллельно учили слабые модели внутри ансамбля. Бустинг в свою очередь является подходом, когда ансамбль составляется последовательно. Слабая модель дополняется другой слабой моделью так, что вторая корректирует ошибки первой - таким образом набирается набор слабых моделей, каждая из которых корректирует предсказания предыдущих.

> Самым первым подходом бустинга является алгоритм Adaboost, при этом рассматривать мы его не будем. Вы можете ознакомиться с данным подходом на различных ресурсах интернет. Например [здесь](https://www.youtube.com/watch?v=LsK-xG1cLYA).

# Сразу переходим к практике!

Можно много слов сказать, а мы сразу к делу! Начнем с создания набора данных:

> В данной практике мы начнем с задачи регрессии, так как на ее примере дается лучшее представление того, как алгоритм градиентного бустинга учится и работает.

In [ ]:
X_data = np.linspace(-1, 7, 200)[:, None]
y_data = np.sin(X_data[:,0])*5 + np.random.normal(size=X_data.shape[0])*2 + 5

# Посмотрим на данные
plt.scatter(X_data[:,0], y_data)
plt.grid(True)
plt.xlabel('Значение признака ($x$)')
plt.ylabel('Истинное значение ($y$)')
plt.show()

По рисунку видно, что данные имеют синусоидальную зависимость. Самое время научиться применять алгоритм градиентного бустинга, чтобы модель смогла определить и повторять данную зависимость в данных!

Начнем формирование ансамбля с наиболее простой модели предсказания для регрессии - предсказание среднего значения по вектору истинных значений!

> Помните R2? Эта метрика тоже оценивает, насколько лучше модель работает, чем модель, которая всегда предсказывает среднее значение по вектору разметки!

Так вот, давайте напишем класс модели, которая во время обучения определяет среднее значение разметки и запоминает его. В качестве предсказания он всегда отвечает в виде среднего значения. Поехали!

In [ ]:
# TODO
class MeanPredictor:
    def __init__(self):
        self.y_mean = 0

    def fit(self, X, y):
        # TODO - вот здесь нужно получить среднее по y и сохранить
        self.y_mean = None

    def predict(self, X):
        # TODO - предсказание - вектор по размеру количества записей,
        #   заполненный средним значением, которое мы сохранили при обучении
        return y_pred

In [ ]:
# TEST

X = np.array([
    [0, 1, 2],
    [3, 1, 1],
    [0, 0, 1],
])
y = np.array([0, 1, 2])

mean_model = MeanPredictor()
mean_model.fit(X, y)

y_pred = mean_model.predict(X)

assert len(y_pred.shape) == 1
assert y_pred.shape[0] == 3
assert np.all(y_pred == np.array([1, 1, 1]))

Теперь убедимся на наших данных, что все работает и наша первая слабая модель отлично работает:

In [ ]:
X = X_data
y_true = y_data

weak_mean = MeanPredictor()
weak_mean.fit(X, y_true)

y_pred = weak_mean.predict(X)
print(y_pred[:5])

Превосходно! Теперь самое время отобразить наши данные, предсказания модели и отклонения (ошибки) модели:

In [ ]:
y_pred_0 = weak_mean.predict(X)
y_resid_0 = y_true-y_pred_0

X_vis = np.linspace(X_data.min(), X_data.max(), 100)[:, None]
y_pred_vis = weak_mean.predict(X_vis)

fig, ax = plt.subplots(1, 2, sharey=True)
ax[0].plot(X_vis, y_pred_vis, 'r--', lw=3)
ax[0].scatter(X_data, y_data)
ax[0].set_title('Данные и предсказания')
ax[0].grid(True)

ax[1].scatter(X_data, y_resid_0, color='green')
ax[1].set_title('Отклонения')
ax[1].grid(True)

plt.show()

Можно заметить, что модель действительно предсказывает среднее по даным значение, а отклонения колеблятся вокруг нуля. Для уверенности можем сразу вывести показатель R2, используя реализацию из `sklearn.metrics.r2_score`

In [ ]:
from sklearn.metrics import r2_score

r2_value_0 = r2_score(y_true, y_pred_0)
print(f'R2 score for step 0 = {r2_value_0}')

А еще, чтобы оценивать, как близко отклонения к нулю, мы воспользуемся метрикой MSE (`sklearn.metrics.mean_squared_error`):

In [ ]:
from sklearn.metrics import mean_squared_error as mse_score

mse_value_0 = mse_score(y_true, y_pred_0)
print(f'MSE score for step 0 = {mse_value_0}')

О чем это нам говорит? Если вспомните, то R2 показывает значения в диапазоне [0; 1], если модель работает лучше, чем просто предсказание среднего, и в диапазоне (-inf, 0], если хуже. Соответственно, мы предсказываем среднее значение, поэтому 0!

> Зачем мы называли переменные с суффиксом `*_0`? Это чтобы определить шаг обучения. Да, да, мы уже начали учить модель бустинга! Просто пока разбираем по шагам - будут индексы шагов, а потом все будет объединено!

Продолжение обучения модели бустинга заключается в том, что мы берем отклонения предсказаний первой модели и должны составить такую модель, которая обучится на этих отклонениях в качестве разметки. То есть $y^{<1>} = y-\hat{y}^{<0>}$, где $y^{<1>}$ - разметка для обучения на шаге 1, а $\hat{y}^{<0>}$ - предсказания модели на шаге 0.

По сути, самая первая слабая модель учится на исходных данных, следующая модель обучается на исходных признаках, но вот вектор истинных значений уже не исходный, а вектор отклонений предсказаний первой модели.

> Думаю, и так понятно, что использовать модель, предсказывающую среднюю модель не имеет смысла, так как новая модель учится на отклонениях, а среднее значение отклонений сейчас равно 0. Поэтому бустинг на модель, предсказывающих среднее не получится.

Мы будем использовать в качестве слабой модели (но более сильной по сравнению с простым предсказанием среднего) решающее дерево. Мы раньше знакомились с деревьями для решения задачи классификации, но они также могут применяться и для регресии. Строятся они почти также, просто при делении оценивается не индекс Джини или энтропия, а классический MSE. Поэтому мы не будем заострять на этом внимание и просто воспользуемся реализацией `sklearn.tree.DecisionTreeRegressor`.

Еще одной особенностью, которой мы воспользуемся, является ограничение глубины дерева при построении. Установим максимальную глубину равную единице. А знаете как зовется такое дерево, в котором всего один узел и два листа? Подумайте =)

<details>
<summary>Ответ</summary>

Пенек!
</details>

> В данном случае мы ограничиваем глубину деревьев так сильно, чтобы наблюдать пошаговый эффект. В реальности это может являться одним из гиперпараметров при построении модели, если бустинг делается над решающими деревьями. Вам ведь никто не запрещает делать бустинг над линейными моделями =)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

weak_stump = DecisionTreeRegressor(
    max_depth=1,
    random_state=RANDOM_STATE
)

weak_stump.fit(X_data, y_resid_0)

y_pred_1 = weak_stump.predict(X_data)
y_resid_1 = y_data-(y_pred_0+y_pred_1)

Вот таким нехитрым способом происходит обучение пенька. Так как дерево маленькое, давайте отобразим его структуру:

In [ ]:
from sklearn import tree

tree.plot_tree(weak_stump)

Здесь видно, как узел разделяет значения признака по конкретному значению. Давайте теперь посмотрим на то, как наша вторая слабая модель предсказывает:

In [ ]:
y_pred_1_vis = weak_stump.predict(X_vis)

plt.plot(X_vis, y_pred_1_vis, 'r--', lw=3)
plt.scatter(X_data, y_resid_0, color='green')
plt.grid(True)
plt.title('Отклонения и предсказания пня')
plt.show()

Видно, что модель, хоть и очень плохо, но постаралась предсказать значения, соответсвующие более менее нашим отклонениям.

> Для проверки убедитесь, что информация на картинке соответствует информации в структуре дерева.

Хорошо, мы видим, что пень постарался решить задачу регрессии по отклонениям, но что же с моделью бустинга? У нас есть модель среднего и теперь пенек, что делать дальше?

Ответ очень прост! Просим обе модели сделать предсказание на данных ($X$) и складываем предсказанные значения! Поехали:

In [ ]:
y_pred_0 = weak_mean.predict(X)
y_pred_1 = y_pred_0 + weak_stump.predict(X)

print(y_pred_1[:4])

Такс, а теперь, чтобы убедиться, что стало лучше, посмотрим на значение R2:

In [ ]:
r2_value_1 = r2_score(y_true, y_pred_1)
print(f'R2 score for step 1 = {r2_value_1}')

Таакс, показатели выросли, это говорит о том, что такой подход действительно получился и работает! Но есть небольшая проблемка...

Мы сделали два шага и уже есть два объекта, которые вместе представляют собой ансамбль. То есть эти две слабые модели - это уже модель градиентного бустинга!

А что если мы захотим иметь в ансамбле 100 моделей? Давайте автоматизировать! Для начала сделаем список, который и будет по сути моделью бустинга:

In [ ]:
gb_model = [weak_mean, weak_stump]

А теперь напишем функцию предсказания, которая будет на вход получать модель бустинга (как бы список) и делать предсказания:

> Еще разок, предсказание модели бустинга состоит в том, чтобы взять предсказания всех моделей в ансамбле как суперпозицию (сумму предсказаний) моделей! Почему это работает? - Разберем чуть позже.

In [ ]:
# TODO - напишите функцию предсказания модели бустинга
def predict_gb(gb_model, X):
    '''
    gb_model - список слабых моделей в ансамбле в порядке обучения
    '''
    return y_pred

In [ ]:
# TEST
# Проведем тест прямо на наших данных!

y_pred = predict_gb(gb_model, X_data)

assert y_pred.shape[0] == X.shape[0]
assert len(y_pred.shape) == 1
# Так как мы не добавляли моделей в ансамбль, сейчас предсказания должны сойтись!
assert np.all(y_pred == y_pred_1)

Все, уже проще, нам не нужно делать кучу объектов и вызовов `.predict()` - все делается функцией! Переходим к визуальной оценке - сейчас это наиболее важно для понимания!

И тут чирканем функцию для отображения, я сам напишу =)

In [ ]:
def plot_model(gb_model, X, y): 
    X_vis = np.linspace(X.min(), X.max(), 100)[:, None]
    y_resid = y-predict_gb(gb_model, X)
    y_pred_vis = predict_gb(gb_model, X_vis)

    fig, ax = plt.subplots(1, 2, sharey=True, figsize=[10, 5])
    ax[0].plot(X_vis, y_pred_vis, 'r--', lw=3)
    ax[0].scatter(X, y)
    ax[0].set_title('Данные и модель бустинга')
    ax[0].grid(True)

    ax[1].scatter(X, y_resid, color='green')
    ax[1].set_title('Отклонения')
    ax[1].grid(True)

    plt.show()

In [ ]:
X = X_data
y_true = y_data

plot_model(gb_model, X, y_true)

y_pred = predict_gb(gb_model, X)
mse_value = mse_score(y_true, y_pred)
print(f'Current MSE score: {mse_value}') 

Наконец-то, графики! Что мы тут видим? Модель уже не представляет собой прямую линию, а приближается к данным! И более того, отклонения стали ближе к нулю - MSE стал заметно ниже. О чем это говорит?

Да вот о том, что подход, когда мы берем нынешнее предсказание модели бустинга, вычисляем отклонения и передаем их в качестве целевых значений новой слабой модели на обучение - работает! Давайте для уточнения проведем еще пару шагов, чтобы убедиться, что подход действительно работает. Но перед этим напишем функцию обучения и добавления новой модели в ансамбль:

In [ ]:
# TODO - напишите функцию обучения слабой модели (пня)
def fit_new_weak_model(gb_model, X, y):
    return weak_stump

In [ ]:
# TEST
# Тут мы опять сразу проверим на наших данных

X = X_data
y_true = y_data

new_model = fit_new_weak_model(gb_model, X, y_true)

assert isinstance(new_model, DecisionTreeRegressor)
assert np.isclose(np.mean(new_model.predict(X)), 0)

Отлично! А теперь давайте напишем цикл из 20 итераций, в котором будем обучать нашу модель бустинга и отображать как метрики, так и графики! Поехали:

In [ ]:
X = X_data
y_true = y_data

for i in range(20):
    print(f'Step {i}')
    new_model = fit_new_weak_model(gb_model, X, y_true)
    gb_model.append(new_model)

    y_pred = predict_gb(gb_model, X)

    r2_value = r2_score(y_true, y_pred)
    mse_value = mse_score(y_true, y_pred)

    plot_model(gb_model, X, y_true)
    print(f'R2: {r2_value} | MSE: {mse_value}')
    print('---------')

Ого, сколько графиков! Но давайте пройдемся по ним и обратим внимание на то, что с каждой новой итерацией улучшаются показатели метрик, отклонения все ближе становятся к нулю и модель все больше соответствует зависимости в данных! Это успех!

Так что нам остается понять? Да именно то, что градиентный бустинг так и работает! Пока кратко:
- начинаем с самой простой модели предсказания среднего значения (константа);
- каждая новая модель в ансамбле учится на отклонениях предсказания модели бустинга;
- предсказание модели происходит путем суммирования предсказаний всех моделей (это относится к регрессии).

То есть идея подхода в том, что мы обучили модельку, поняли, насколько отклоняются предсказания от истины и задача новой модельки в ансамбле - подкорректировать/помочь модели путем исправления ошибок. Фактически, **новая модель в ансамбле учится исправлять ошибки ранее обученных моделек**. Так и получается, что мы *бустим* нашу модель, пока не получим норм результат.

И можно сказать, что такая модель шикарно работает и нет ей аналогов и мы нашли инструмент для любого случая! Отчасти - правда, так как на том же кагле разновидности именно такого подхода чаще всего выигрывают =)

Но и тут есть ложка дегтя =( А выяснить в чем дело мы предлагаем вам! 

## Задание

Попробуйте разделить наш датасет на два набора (обучение и валидация) и обучить модель на 500 итераций, при этом сохранять значение R2 на каждой итерации и потом отобразить график. Посмотрите, что происходит с графиком R2 и напишите выводы.

In [ ]:
# TODO - вот здесь пишем код разделения данных, обучения,
#   оценки и отображения графика R2



# NOTE - создайте новую модель, не используйте старый gb_model
gb_model = [] 
r2_values_train = []
r2_values_test = []

# А вот здесь пишем код создания новых пеньков и обучения =)
# Не забудьте заносить в список R2 метрику для тестовых данных! 


# Визуализация обучения - уже написано
plt.figure(figsize=[10, 10])
plt.plot(r2_values_train, 'b', label='Train')
plt.plot(r2_values_test, 'k', label='Test')
plt.grid(True)
plt.legend()
plt.show()

TODO - А вот здесь пишем выводы!



<details>
<summary>Подсказка</summary>

Если вы не заметили переобучения, посмотрите еще разок =)
</details>

# Формальная часть вопроса

Когда мы поняли всю идею подхода, пора чуток коснуться, откуда корни растут! Еще раз спасибо великой математике за это!

> Мы тут предсказание модели обозначим через $h(X)$, чтобы не путать с параметрами $W$ все время.

## Причем тут градиенты?

Начнем наш рассказ с воспоминаний о том, что очень тесно связано с обучением, когда речь идет о методах оптимизации (градиентном спуске) - функции потерь! Фактически, это метрика, но только она обязана быть дифференцируемой. Так как у нас задача регрессии, то метрика как обычно $MSE$ (и как обычно, в виде функции потерь с двоечкой):

$$
J(\hat{y}) = MSE = \frac{1}{2*n}\sum_{n} (y-\hat{y})^2
$$

> А вы тоже заметили, что термин *отклонения* по ходу обучения бустинга и термин *отклонения* в "*сумма квадратов отклонений*" очень похожи?

Так вот вы не поверите, но все эти действия, что мы делали до этого - мы учили модель градиентным спуском! Давайте начнем как начинали и вспомним самую первую модель, почему мы выбрали именно среднее?

По сути, задачу обучения модели можно записать так:
$$
h(X) = {argmin}_\hat{y} J(\hat{y})
$$
> В этом обозначении мы стараемся найти *такое* предсказание $\hat{y}$, чтобы получить *минимум функции потерь*.

то есть, предсказания модели должны соответствовать минимуму функции потерь. Мы в качестве самой простой модели (самой первой) хотим предсказывать константу, поэтому функция потерь будет выглядеть так:
$$
J(\hat{y}) = \frac{1}{2*n} [(y^{(1)}-\hat{y}^{(1)})^2 + (y^{(2)}-\hat{y}^{(2)})^2 + \dots + (y^{(n)}-\hat{y}^{(n)})^2]
$$

Так как предсказание - константа, то $\hat{y}^{(1)} = \hat{y}^{(2)} = \dots = \hat{y}^{(n)}$. Все сводится к тому, чтобы найти $\hat{y}$, которое даст минимум функции потерь. Давайте построим график на наших данных:

In [ ]:
y_pred_vis = np.linspace(2, 7, 30)
loss_values = []

for y_pred_val in y_pred_vis:
    loss_value = mse_score(y_true, np.full_like(y_true, y_pred_val))/2
    loss_values.append(loss_value)

plt.figure(figsize=[10, 7])
plt.plot(y_pred_vis, loss_values)
plt.grid(True)
plt.ylabel('Loss ~ $J(\hat{y})$')
plt.xlabel('$\hat{y}$')
plt.show()


То есть по графику мы видим, что в зависимости от значения константного предсказания мы получаем то или иное значение функции потерь. Но как найти минимум? Причем мы меняем всего одно число. Эврика! Найдем производную функции потерь и приравняем ее к нулю! Поехали!

$$
\frac{\partial}{\partial \hat{y}} J(\hat{y}) = 
\frac{\partial}{\partial \hat{y}} \frac{1}{2*n}\sum_{n} (y-\hat{y})^2 = 
\frac{1}{2*n}\sum_{n} \frac{\partial}{\partial \hat{y}} (y-\hat{y})^2
$$

Осталось определить производную квадрата отклонений:
$$
\frac{\partial}{\partial \hat{y}} (y-\hat{y})^2 = 2*(y-\hat{y}) \frac{\partial}{\partial \hat{y}} (y-\hat{y}) = -2*(y-\hat{y})
$$

В результате получаем итог:
$$
\frac{\partial}{\partial \hat{y}} J(\hat{y}) = 
\frac{-1}{n}\sum_{n} (y-\hat{y})
$$

> Обратите внимание, мы тут не по весам производную ищем, а по значению предсказываемой константы =)

Так значит лучшая константа, которая выбирается для начала создания модели будет:
$$
\frac{\partial}{\partial \hat{y}} J(\hat{y}) = 
\frac{-1}{n} [(y^{(1)}-\hat{y}) + (y^{(2)}-\hat{y}) + \dots 
+ (y^{(n)}-\hat{y})] = 0
$$

А это приводит нас выводу, что:
$$
\frac{1}{n} [y^{(1)} + y^{(2)} + \dots 
+ y^{(n)}] = \hat{y}
$$

> Если что, так как все значения предсказания равны константе, я просто перенес все вправо.

Воу, так вот как получилось, что лучше всего в качестве начальной модели выбрать среднее по истинным значениям? Круто!

А еще мы вычислили градиент функции потерь по значению предсказания, что тоже очень полезно!

## Продолжаем мучать градиент

Но на этом наш разбор не закончен, мы только разобрались, почему среднее значение в качестве первой модели - лучший вариант. Пора перейти к вопросу, как так вышло, что новые модели учатся на отклонениях старых?

Вводная, мы уже поняли, как получить модель $h^0(X)$, так что сейчас сидим и думаем, что делать на этом шаге.

Раньше мы пользовались градиентом, когда хотели менять веса в модели, чтобы уменьшить ту же самую функцию потерь. Выглядело это все таким образом:
$$
W \leftarrow W - \alpha \frac{\partial J(W)}{\partial W}
$$

То есть градиентом вы выясняли, как надо поменять веса модели, чтобы уменьшить значение функции потерь (а значит улучшить работу модели) и обновляли веса. 

Теперь давайте подумаем над этим обозначением:
$$
\frac{\partial J(h(X))}{\partial h(X)}
$$

Такая производная говорит нам о том, как надо **поменять модель**, чтобы изменить значение функции потерь. По сути, можно было бы сделать так:
$$
h(X) \leftarrow h(X) - \frac{\partial J(h(X))}{\partial h(X)}
$$

> В формуле опущен $\alpha$, мы его обсудим позже.

Получается, что для улучшения нам надо просто прибавить какие-то числа? Не совсем так просто. Когда мы обновляли веса, которые являются числами, то мы и прибавляли числа. Тут - **модели**, а это не числа, а функции/алгоритмы и их не просто "вычислить" как числа. 

Но не все потеряно! Мы же можем обучить новую модель, которая как раз и будет являться тем, что мы можем отнимать/прибавлять по отношению к другим моделям! То есть, нашей задачей является обучить модель предсказывать $-\frac{\partial J(h(X))}{\partial h(X)}$!

Подводя итог, если мы обучим некоторую модель $s^1(X)$ предсказывать значения $-\frac{\partial J(h(X))}{\partial h(X)}$, то потом можем сделать так:
$$
h^1(X) = h^0(X) + s^1(X)
$$

и получить новую модель, которая (по идее) должна быть лучше!

Таакс, а теперь самое сладкое! Вспоминаем, что производная функции потерь $MSE$ по предсказываемому значению - это просто отрицательный остаток:
$$
-\frac{\partial J(h(x))}{\partial h(x)} = (y-h^0(X))
$$
> Тут нет знака суммы, потому что мы сейчас рассматриваем каждую запись отдельно.

Вот таким вот образом новая модель $s^1(X)$ пытается обучиться предсказывать остатки, чтобы в результате добавления этой модели в ансамбль это уменьшило функцию потерь!

Вот такая история!

## То, о чем не поговорили

И на этом моменте появляется **небольшое признание**, после обучения новой модели, чтобы ее прибавить к старым, делается еще один шаг. Принцип построения и предсказания остается примерно таким же. При добавлении модели мы делаем не так:
$$
h^1(X) = h^0(X) + s^1(X)
$$

а вот так:
$$
h^1(X) = h^0(X) + \alpha * s^1(X)
$$


$\alpha$ - это **коэффициент модели**! Этот коэффициент вычисляется путем минимизации по следующему принципу:
$$
\alpha = {argmin}_\alpha J(h^0(X)+\alpha*s^1(X))
$$

То есть нам надо подобрать такой корректирующий коэффициент для новой модели, который будет давать минимум функции потерь с ним.

> Для дерева мы каждому листу свой коэффициент присваиваем.

В ходе обучения слабой модели (решающее дерево для регрессии) используется функция потерь MSE, как и для обучения всего бустинга. Этот коэффициент требуется, когда алгоритмы используют разные функции потерь для обучения.

> Для данного случая можете проверить - коэффициент будет все время равен единице.

Давайте разберемся, для какого случая это будет применимо. Что если наша функция потерь не MSE, а MAE:
$$
J(h(X)) = |y-h(X)|
$$

Тогда отрицательная производная этой функции будет следующая:
$$
-\frac{\partial J(h(X))}{\partial h(X)} = sign(y-\hat{y})
$$

То есть производная равна +1 или -1. В таком случае "отклонения" будут выглядеть так:

In [ ]:
def plot_model_mae(gb_model, X, y):
    X_vis = np.linspace(X.min(), X.max(), 100)[:, None]
    y_resid = np.sign(y-predict_gb(gb_model, X))
    y_pred_vis = predict_gb(gb_model, X_vis)

    fig, ax = plt.subplots(1, 2, sharey=True, figsize=[10, 5])
    ax[0].plot(X_vis, y_pred_vis, 'r--', lw=3)
    ax[0].scatter(X, y)
    ax[0].set_title('Данные и модель бустинга')
    ax[0].grid(True)

    ax[1].scatter(X, y_resid, color='green')
    ax[1].set_title('Отклонения')
    ax[1].grid(True)

    plt.show()

In [ ]:
X = X_data
y_true = y_data

start_model = MeanPredictor()
start_model.fit(None, y_true)
gb_model = [start_model]

plot_model_mae(gb_model, X, y_true)

Как видите, отклонения уже не совсем "отклонения", а просто знак отклонения, поэтому эта часть, когда мы вычисляем отрицательную производную функции потерь, называется **псевдо-остатки**. Реальными остатками они являлись, когда мы использовали MSE, но такое случается не всегда, поэтому и появляется приставка *псевдо*.

Теперь, зачем тут коэффициент $\alpha$? А посмотрите, когда новая модель будет учиться на псевдо-остатках, то они не будут видеть истинных значений отклонений, а только знак. Тогда модель научится определять сторону, в которую нужно направлять коррекцию, но не будет знать насколько. Для этого и добавляется коэффициент модели (в случае дерева - каждому листу), чтобы подкорректировать *масштаб* предсказаний коррекции.

In [ ]:
def fit_new_weak_model_mae(gb_model, X, y):
    y_pred = predict_gb(gb_model, X)
    y_resid = np.sign(y-y_pred)

    weak_stump = DecisionTreeRegressor(
        max_depth=1,
        random_state=RANDOM_STATE
    )

    weak_stump.fit(X, y_resid)

    return weak_stump

In [ ]:
X = X_data
y_true = y_data

start_model = MeanPredictor()
start_model.fit(None, y_true)
gb_model_mae = [start_model]

new_model = fit_new_weak_model_mae(gb_model_mae, X, y_true)
gb_model_mae.append(new_model)

plot_model_mae(gb_model_mae, X, y_true)

In [ ]:
# А теперь для примера отобразим обучение с MSE
start_model = MeanPredictor()
start_model.fit(None, y_true)
gb_model_mse = [start_model]

new_model = fit_new_weak_model(gb_model_mse, X, y_true)
gb_model_mse.append(new_model)

plot_model(gb_model_mse, X, y_true)

Как видите, модель с функцией отличной от MSE (попробовали MAE) обучилась на остатках и размах результата не такой, как в случае использования MSE. Вся причина в том, что при использовании MSE остатки имеют масштаб не +/- 1, а полноценные ошибки, так что это помогает модели обучаться.

К чему это все? Функция потерь была взята MAE для примера, но при этом часто функция может быть совсем другой (не MSE), поэтому надо добавлять коэффициент каждой новой модели, чтобы мастабировать до корректных значений.

Здесь мы коэффициент искать не будем, так как это отдельный случай, но вы можете попробовать сделать это сами или использовать готовые инструменты реализации бустинга.

На этом моменте мы попробовали и освоили алгоритм градиентного бустинга в базовом варианте! 

* Обратите внимание, что модель бустинга отлично подстраивается под данные, при этом не имея регуляризации, очень подвержена переобучению!

* Тем не менее данный ансамблевый метод является очень распространенным и широко используется в работе с данными!

Главное, как и с любым другим инструментом, понимать, как он работает, а уже с конкретными реализациями и их тонкосятми (как, например, [CatBoost](https://catboost.ai), [XGBoost](https://xgboost.readthedocs.io/en/latest/), [LightGBM](https://lightgbm.readthedocs.io/en/latest/)) можно разобраться уже по ходу дела! Превосходно!

# Еще ресурсы

Мы неплохо постарались и разобрались с основами градиентного бустинга! И всё-таки, знаниям нет предела и я крайне рекомендую ознакомиться с, например, отличными видео с объяснениями и рассказом по этой теме от StatQuest:
- Gradient Boostring - Regression [part 1](https://www.youtube.com/watch?v=3CC4N4z3GJc), [part 2](https://www.youtube.com/watch?v=2xudPOBz-vs)
- Gradient Boostring - Classification [part 3](https://www.youtube.com/watch?v=jxuNLH5dXCs), [part 4](https://www.youtube.com/watch?v=StWY5QWMXCw)

# Понижение размерности данных

Раз уж заговорили о серьезном, то и обсудим сразу интересную методику под названием *понижение размерности*, которую можно активно применять в решениях задач! Это небольшой шаг в сторону от бустинга, но тем не менее полезно знать и использовать!

Работа с реальными данными часто вынуждает анализировать множества признаков, более двух, что не позволяет отобразить данные графически и сделать визуальную оценку. С одной стороны, большое количество признаков позволяет делать прогнозы более точно, а расширение количества признаков за счет новой информации или feature engineering может положительно сказаться на предиктивной способности модели.

> Предиктивная способность модели - объединение обобщающей способности модели с ее точностью предсказания. Можно назвать *корректность модели*.

Но ложкой дегтя здесь является то, что не все модели положительно реагируют на слишком большое количество признаков. Так, например, деревья, деля данные лишь по одному признаку за раз, не могу создавать обобщающие гипотезы среди нескольких признаков, как, например, это делает линейная регрессия. Поэтому, мы рассмотрим подход на основе *обучения без учителя*, который позволяет из набора признаков сделать меньший набор признаков с как можно меньшими потерями информации.

Метод Principal Component Analysis (PCA) является одним из методов понижения размерности. Все методики понижения размерности нацелены на уменьшение количества признаков путем выделения новых признаков (чаще всего не имеющих физического представления) меньшего количества.

Метод PCA основан на принципе нахождения новых ортогональных векторов, на которые проецируются данные.

![Пример PCA](https://dinhanhthi.com/img/post/ML/PCA/pca-1.jpg)

Например в случае картинки для двух признаков (2D) ищется такой вектор, чтобы проекция на него имела наименьшую ошибку. Таким образом происходит понижение размерности от двух признаков к одному (2D -> 1D).

Такая линия напоминает работу с линейной регресией, не так ли? Все верно, это очень похоже! Только в качестве метрики для линейной регрессии мы старались сделать расстояние от линии до точек минимальным, а так как $y-\hat{y}$, то расстояние от точки до линии было вертикальной линией - это называется *отклонение предсказания / ошибка предсказания*. Пример на картинке ниже:


In [ ]:
x = np.linspace(-1.2,1.2,20)
y = x
dy = (np.random.rand(20)-0.5)

plt.plot(x,y)
plt.scatter(x,y+dy)
plt.vlines(x,y,y+dy)
plt.ylabel('$y$')
plt.xlabel('$x$')

**Ошибка проекции** - это тоже расстояние от точек до линии, но только это расстояние отсчитывается наикратчайшим, то есть перпендикулярным вектору, на который делается проекция (как показано на картинке про PCA).

То есть мы из двух признаков делаем проекцию на вектор и получаем один (вместо двух). Так последовательно сводя по два признака к одному мы уменьшаем размерность данных (количество признаков), при этом зачастую теряя их физический смысл.

> Суть в том, что найдя вектор PCA между "площадью дома" и "количеством соседей" в данных, мы можем попытаться выразить это через какое-то взаимосвязанное определение, но чаще всего это проще назвать PCA компонентой.

Чем полезен такой подход? Помимо того, что у нас появляется возможность отобразить многомерные данные, также мы можем получить улучшение работы модели, так как мы уменьшаем влияние эффекта под названием **проклятье размерности**.

> Проклятье размерности (Curse of Dimensionality) - термин, который описывает сложности работы с данными, в которых [слишком много признаков](http://www.machinelearning.ru/wiki/index.php?title=%D0%9F%D1%80%D0%BE%D0%BA%D0%BB%D1%8F%D1%82%D0%B8%D0%B5_%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%80%D0%BD%D0%BE%D1%81%D1%82%D0%B8).

Давайте теперь посмотрим, как работать с этим методом в `sklearn`. Для примера возьмем датасет по классификации вин:

In [ ]:
from sklearn.datasets import load_wine

wine_data = load_wine()
feat_names = wine_data['feature_names']

df = pd.DataFrame(wine_data['data'], columns=feat_names)
df['CLASS'] = wine_data['target']

df.head()

In [ ]:
from sklearn.decomposition import PCA

X_df = df[feat_names]
y = df['CLASS']
target_names = [
    'Class0',
    'Class1',
    'Class2',
]

# Задается количество признаков, которое хотим получить
# Для визуализации понижаем количество признаков до 2х
pca = PCA(n_components=2)
pca.fit(X_df)
X_pca = pca.transform(X_df)

plt.figure()
for l, c, m in zip(range(0, 3), ('blue', 'red', 'green'), ('^', 's', 'o')):
    plt.scatter(
        X_pca[y == l, 0],
        X_pca[y == l, 1],
        color=c,
        label=target_names[l],
        alpha=0.8,
        marker=m
    )

plt.legend(target_names)
plt.xlabel('PCA_component_0')
plt.ylabel('PCA_component_1')
plt.grid()

Таким вот образом мы смогли отобразить данные, которые представляют собой 13 признаков. Мы видим, что Class0 хорошо отделим от остальных данных, а остальные два класса сильно смешались. 

# Задание

Попробуйте произвести стандартизацию данных и после этого проверьте, как работает метод PCA после стандартизации (отобразите данные):

In [ ]:
# TODO

В результате видно, что стандартиация данных очень положительно сказывается на работе метода PCA. После стандартизации и понижения размерности данные стали намного более разделимы.

# Задание

Оцените кросс-валидацией показатель `f1_macro` трех вариантов на данных:
- Модель логистической регрессии;
- Модель логистической регрессии со стандартизацией;
- Модель ЛР после понижения размерности с помощью [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) до 2х компонент;
- Модель ЛР со стандартиацией и понижением размерности до 2х компонент;
- Модель случайного леса;
- Модель случайного леса со стандартиацией и понижением размерности до 2х компонент;


In [ ]:
# TODO

# Задание

Оцените работу модели Случайного леса со стандартизацией и понижением размерности до количества в диапазоне [2; 13] (всего 12 значений); постройте график в осях (количество компонент; показатель метрики на кросс-валидации);


In [ ]:
# TODO

# Задание

Разберитесь с методом [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) и сравните влияние стандартизации и работу с методом PCA - визуализируйте данные и проверьте влияние стандартизации.

In [ ]:
# TODO

# Задание - выводы

Как всегда, ваши выводы - это одна из самых ценных вещей для вас самих, не стесняйтесь записывать даже самые малые наблюдения и умозаключения! Мы вам постараемся помочь вопросами:

- Чем отличается принцип работы PCA и tSNE? В чем их назначения? Можно ли оба использовать для понижения размерности?
- Какие еще существуют методы понижения размерности?
- Чем бустинг отличается от бэгинга?
- Почему бустинг - это ансамблевый метод?
- Можно ли построить модель бустинга на основе случайных лесов? (Бустинг над бэггингом)? Будет ли в этом смысл?

# Вопросы

1. Почему бустинг является методом ансамблирования? 
2. Что показывает градиент? 
3. Зачем нужен коэффициент модели? 
4. Зачем понижать размерность данных? 
5. Какая структура "растительности" используется в AdaBoost? 
6. Почему модель может быть названа слабой? 

# Небольшое заключение

На данный момент мы освоили уже некоторое (хорошее) количество различных моделей:
- Линейная/логистическая регрессия;
- (Если выполняли лабу) KNN/SVM;
- Решающие деревья/случайный лес;
- Градиентный бустинг.

Мы на этом не останавливаемся, но нужно сделать небольшое резюме! Простые модели отлично подходят для решения простых задача, потому что работа с ними имеет меньше шансов на переобучение и другие проблемы. При работе со сложными задачами лучше использовать леса или бустинг. Именно поэтому первым делом **всегда** необходимо анализовать имеющиеся данные и искать возможности их улучшить/очистить/исправить/дополнить. Хорошо обработанные данные могут дать выше процент точности с использованием логистической регрессии, нежели сырые данные с применением леса! Такое случается не всегда, но важно понимать, что данные в большинстве определяют способность модели предсказывать.

После того, как данные подготовлены, можно попробовать использовать различные модели и поискать гиперпараметры, которые дадут наилучшую точность. Вот здесь важно помнить об **обобщающей способности модели** и не влететь в переобучение на радостях высоких процентов точности!

По результатам работы всегда возвращайтесь к анализу ошибок модели, так как это еще один источник информации! Те же леса могут в явном виде оценивать важности признаков, что позволит сконцентрировать внимание и найти еще дополнительной информации в данных!

Теперь о бустинге и бэггинге. Оба подхода являются ансамблевыми, но при этом бустинг своим способом обучения может более точно описывать данные (инкрементно стараемся все больше улучшить модель). Вы в этой практике уже смогли переобучить модель, поэтому важно помнить, что **бустинг подвержен переобучению** больше, нежели леса и подход бэггинга!

Так что при разработке модели осознанный выбор лучше делать простым испытанием обоих подходов! Просто берем и ищем лучшие модели обоих подходов и после уже решаем, что в данном случае подходит больше. А для сравнения моделей мы уже узнали про различные показатели метрик. Помимо этого, на принятие решений может повлиять и специфика решаемой задачи.

Не бойтесь экспериментировать и узнавать новое! Как правило классические подходы дают хорошие результаты предсказаний, а лучшие достигаются крайне нестандартными способами - творите!